# Ollama + OpenAI + Python

## 1. Določite ime modela

Če ste uporabili drugačen model kot "phi3:mini", spremenite vrednost v spodnji celici. Ta spremenljivka bo uporabljena v kodi skozi celoten zvezek.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Nastavitev Open AI odjemalca

Običajno se OpenAI odjemalec uporablja z OpenAI.com ali Azure OpenAI za interakcijo z velikimi jezikovnimi modeli.  
Vendar pa ga je mogoče uporabiti tudi z Ollama, saj Ollama ponuja OpenAI-kompatibilno končno točko na "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Ustvari dokončanje klepeta

Zdaj lahko uporabimo OpenAI SDK za ustvarjanje odgovora na pogovor. Ta zahteva naj ustvari haiku o mačkah:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Oblikovanje pozivov

Prvo sporočilo, poslano jezikovnemu modelu, se imenuje "sistemsko sporočilo" ali "sistemski poziv" in določa splošna navodila za model.  
Lahko podate svoj prilagojen sistemski poziv, da usmerite jezikovni model k ustvarjanju izhoda na drugačen način.  
Spremenite `SYSTEM_MESSAGE` spodaj, da model odgovarja kot vaš najljubši lik iz filma/TV serije, ali pa poiščite navdih za druge sistemske pozive na [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Ko prilagodite sistemsko sporočilo, podajte prvo uporabniško vprašanje v `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Primeri z nekaj primeri

Drug način za usmerjanje jezikovnega modela je, da mu podamo "nekaj primerov", torej zaporedje vprašanj in odgovorov, ki pokažejo, kako naj odgovarja.

Spodnji primer poskuša jezikovni model spodbuditi, da deluje kot asistent pri poučevanju, tako da poda nekaj primerov vprašanj in odgovorov, ki bi jih lahko dal asistent, nato pa modelu zastavi vprašanje, ki bi ga lahko postavil študent.

Najprej poskusite, nato pa prilagodite `SYSTEM_MESSAGE`, `EXAMPLES` in `USER_MESSAGE` za nov scenarij.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Pridobivanje z okrepljeno generacijo

RAG (Pridobivanje z okrepljeno generacijo) je tehnika, ki omogoča jezikovnemu modelu, da natančno odgovarja na vprašanja za določen domen, tako da najprej pridobi ustrezne informacije iz vira znanja in nato ustvari odgovor na podlagi teh informacij.

Na voljo imamo lokalno CSV datoteko z podatki o hibridnih avtomobilih. Spodnja koda prebere CSV datoteko, poišče ujemanja z uporabnikovim vprašanjem in nato ustvari odgovor na podlagi najdenih informacij. Upoštevajte, da bo to trajalo dlje kot kateri koli prejšnji primeri, saj modelu pošlje več podatkov. Če opazite, da odgovor še vedno ni utemeljen na podatkih, lahko poskusite z inženiringom sistema ali z drugimi modeli. Na splošno je RAG bolj učinkovit bodisi z večjimi modeli bodisi z natančno prilagojenimi različicami SLM-jev.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne odgovarjamo za morebitna nesporazumevanja ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
